In [1]:
# Modify sys.path
import os
import sys

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)

In [5]:
import numpy as np
import pandas as pd
import os
import pickle
from platypus import NSGAII, NSGAIII, MOEAD, IBEA, EpsMOEA, SPEA2, EpsNSGAII, ProcessPoolEvaluator, unique, nondominated

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, InputLayer
import keras

import config.config as config
from src.data_processing import read_arff, preprocess_data
from src.utils import get_correlation_rank, get_relieff_rank, get_dataset_best_H
from src.evaluation import predictions_h_stepsahead_LSTM, predictions_h_stepsahead, calculate_H, train_evaluate_lstm_model

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [6]:
# Load dataset
DATA_PATH = os.path.join('..', 'data', config.DATASET_NAME)

dataset = read_arff(DATA_PATH)
df_dict = preprocess_data(dataset)

train_X_timeseries, train_Y_timeseries, val_X_timeseries, val_Y_timeseries, test_X_timeseries, test_Y_timeseries = df_dict['timeseries']
train_X, train_Y, val_X, val_Y, test_X, test_Y = df_dict['normalized']

In [7]:
normCorr = get_correlation_rank(train_X, train_Y, config.N_ATTRIB)
normCorr

array([0.75292699, 0.69818729, 0.66864659, 0.66411017, 0.66814516,
       0.69271328, 0.71620377, 0.63555951, 0.62244178, 0.61904289,
       0.60594714, 0.60390417, 0.60293474, 0.6002923 , 0.04118615,
       0.08195428, 0.10088643, 0.10571791, 0.09896909, 0.08953424,
       0.08362727, 0.02712769, 0.02170817, 0.0162152 , 0.01095165,
       0.00290207, 0.        , 0.0020042 , 0.41430049, 0.39551417,
       0.40469499, 0.41814493, 0.43251645, 0.43883484, 0.43655216,
       0.96538729, 0.90203268, 0.86300364, 0.84640051, 0.84601153,
       0.86921518, 0.89747104, 0.52586631, 0.51870544, 0.50585368,
       0.51271103, 0.51568939, 0.50498901, 0.52337983, 0.45775237,
       0.44764813, 0.41877857, 0.40116346, 0.39146601, 0.39959042,
       0.41546856, 0.1217908 , 0.11417583, 0.10720709, 0.09419372,
       0.07697071, 0.07487274, 0.07735718, 0.33260451, 0.35475143,
       0.35153556, 0.38574562, 0.38134687, 0.34656081, 0.35937949,
       0.33332293, 0.31314523, 0.30289307, 0.28827314, 0.29465

In [8]:
reliefFscores, normScores = get_relieff_rank(train_X, train_Y, config.N_ATTRIB, 10, 5)
normScores

array([0.37073702, 0.29176552, 0.23410287, 0.22844917, 0.21653797,
       0.24370375, 0.29131188, 0.33536514, 0.31776884, 0.31500573,
       0.30245029, 0.33107789, 0.33208918, 0.33379121, 0.23649272,
       0.2106134 , 0.1882    , 0.18017486, 0.16274574, 0.14549386,
       0.19935742, 0.47141279, 0.49291365, 0.49656375, 0.4800984 ,
       0.49706054, 0.52403116, 0.51545195, 0.1787034 , 0.18338228,
       0.20576584, 0.17508201, 0.16236258, 0.1610362 , 0.17039297,
       0.90114231, 0.77046272, 0.69211493, 0.6448192 , 0.6782723 ,
       0.66827977, 0.72748102, 0.1217787 , 0.15250511, 0.15647354,
       0.16580466, 0.17915019, 0.16825289, 0.17242725, 0.19987993,
       0.21268644, 0.22877694, 0.23639351, 0.23161944, 0.23821354,
       0.20325846, 0.23115608, 0.23591647, 0.25820953, 0.25194385,
       0.28934189, 0.30443629, 0.30639268, 0.06343318, 0.08035382,
       0.09579577, 0.08098701, 0.00456963, 0.        , 0.01405112,
       0.12111851, 0.12059093, 0.11601643, 0.09344161, 0.13023

## MOEA

In [9]:
from problems.FS_O1O2O3_LSTM import *

In [ ]:
if __name__ == "__main__":
    results = OrderedDict()
    
    # define the problem definition
    problem = FS_O1O2O3_LSTM(nVar=config.N_ATTRIB, nobjs=3, test_X=val_X, test_y=val_Y, normCorr=tuple(normCorr))
    
    # instantiate the optimization algorithm to run in parallel
    for alg in ['NSGAII', 'NSGAIII', 'MOEAD', 'IBEA', 'EpsMOEA', 'SPEA2', 'EpsNSGAII']:
        start_time = time.time()
        print(alg)
        results[alg] = {}
        results[alg]['FS_O1O2O3_LSTM'] = []
        for s in range(config.N_SEEDS):
            print("Run", s)
            random.seed(s)
            with ProcessPoolEvaluator(config.N_JOBS) as evaluator:
                if (alg == 'NSGAII'):
                    algorithm = NSGAII(problem, evaluator=evaluator)
                    algorithm.run(config.N_EVAL)
                elif (alg == 'NSGAIII'):
                    algorithm = NSGAIII(problem, divisions_outer=4, evaluator=evaluator)
                    algorithm.run(config.N_EVAL)
                elif (alg == 'MOEAD'):
                    algorithm = MOEAD(problem, weight_generator=normal_boundary_weights, divisions_outer=4, evaluator=evaluator)
                    algorithm.run(config.N_EVAL)
                elif (alg == 'IBEA'):
                    algorithm = IBEA(problem, evaluator=evaluator)
                    algorithm.run(config.N_EVAL)
                elif (alg == 'EpsMOEA'):
                    algorithm = IBEA(problem, epsilons=[0.05], evaluator=evaluator)
                    algorithm.run(config.N_EVAL)
                elif (alg == 'SPEA2'):
                    algorithm = SPEA2(problem, evaluator=evaluator)
                    algorithm.run(config.N_EVAL)
                elif (alg == 'EpsNSGAII'):
                    algorithm = EpsNSGAII(problem, epsilons=[0.05], evaluator=evaluator)
                    algorithm.run(config.N_EVAL)
                    
                results[alg]['FS_O1O2O3_LSTM'].append(algorithm.result)
            
        
        print("--- %s minutes ---" % ((time.time() - start_time)/60))

NSGAII
Run 0
--- 0.16128859519958497 minutes ---
NSGAIII
Run 0
--- 0.1650235414505005 minutes ---
MOEAD
Run 0
--- 0.17444427808125815 minutes ---
IBEA
Run 0
--- 0.1589119036992391 minutes ---
EpsMOEA
Run 0
--- 0.1581820805867513 minutes ---
SPEA2
Run 0
--- 0.16068671544392904 minutes ---
EpsNSGAII
Run 0
--- 0.15826662381490073 minutes ---


In [ ]:
# Save results
with open(rf'../Variables/{config.DATASET_SAVE_NAME}-results-LSTM-O1O2O3-7algorithms.pickle', 'wb') as f:
     pickle.dump([results], f)

## Best FS models from algorithms

In [ ]:
# Load results
with open(rf'../Variables/{config.DATASET_SAVE_NAME}-results-LSTM-O1O2O3-7algorithms.pickle', 'rb') as f:
    results7alg = pickle.load(f)
results7alg = results7alg[0]

In [ ]:
def bestModelsLSTM(T_X, T_y, V_X, V_y, results, algorithms, num_objectives, problem_name, colNames=None):
    if colNames is None:
        colNames = ['Run', 'Algorithm', 'RMSE MOEA', 'N', 'Correlation', 'ReliefF', 'RMSE F Train', 'RMSE F Test', 
                    'RMSE StepsAhead', 'MAE StepsAhead', 'CC StepsAhead', 'RMSE MeanStepsAhead', 
                    'MAE MeanStepsAhead', 'CC MeanStepsAhead', 'SelectedAttrib']

    dfSolutions = pd.DataFrame(columns=colNames)

    for alg in algorithms:
        print(f"Processing algorithm: {alg}")
        
        for run, result in enumerate(results[alg][problem_name]):
            print(f"Run: {run}")
            solutions = unique(nondominated(result))
            
            for sol in solutions:
                select = [var[0] for var in sol.variables]
                num_selected = int(sol.objectives[1])

                entry = {
                    'Run': run,
                    'Algorithm': alg,
                    'RMSE MOEA': sol.objectives[0],
                    'N': num_selected,
                    colNames[4]: sol.objectives[2] if num_objectives >= 3 else np.nan,
                    'ReliefF': sol.objectives[3] if num_objectives == 4 else np.nan,
                    'RMSE F Train': np.nan,
                    'RMSE F Test': np.nan,
                    'RMSE StepsAhead': np.nan,
                    'MAE StepsAhead': np.nan,
                    'CC StepsAhead': np.nan,
                    'RMSE MeanStepsAhead': np.nan,
                    'MAE MeanStepsAhead': np.nan,
                    'CC MeanStepsAhead': np.nan,
                    'SelectedAttrib': T_X.columns[select].to_numpy()
                }

                if num_selected > 0:
                    train_X_selectedA = T_X.iloc[:, select].copy()
                    test_X_selectedA = V_X.iloc[:, select].copy()
                    
                    modelF = Sequential([
                        InputLayer(shape=(1, train_X_selectedA.shape[1])),
                        LSTM(units=config.N_NEURONS, activation='relu', return_sequences=True),
                        Dropout(0.2),
                        Dense(1, activation="linear")
                    ])

                    modelF.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer='adam', 
                                   metrics=[tf.keras.metrics.RootMeanSquaredError()])

                    train_X_reshaped = train_X_selectedA.to_numpy().reshape(train_X_selectedA.shape[0], 1, -1)
                    test_X_reshaped = test_X_selectedA.to_numpy().reshape(test_X_selectedA.shape[0], 1, -1)

                    modelF.fit(train_X_reshaped, T_y, batch_size=config.BATCH_SIZE, epochs=config.EPOCHS, verbose=0)
                    
                    predTrain = modelF.predict(train_X_reshaped, verbose=0).ravel()
                    predTest = modelF.predict(test_X_reshaped, verbose=0).ravel()
                    
                    entry['RMSE F Train'] = root_mean_squared_error(T_y, predTrain)
                    entry['RMSE F Test'] = root_mean_squared_error(V_y, predTest)
                    
                    dfResultados, _, _ = predictions_h_stepsahead_LSTM(test_X_selectedA, V_y, modelF, config.N_STEPS)
                    
                    entry.update({
                        'RMSE StepsAhead': np.round(dfResultados['RMSE'].values, 6),
                        'MAE StepsAhead': np.round(dfResultados['MAE'].values, 6),
                        'CC StepsAhead': np.round(dfResultados['CC'].values, 6),
                        'RMSE MeanStepsAhead': np.round(dfResultados.iloc[1:, :]['RMSE'].mean(), 6),
                        'MAE MeanStepsAhead': np.round(dfResultados.iloc[1:, :]['MAE'].mean(), 6),
                        'CC MeanStepsAhead': np.round(dfResultados.iloc[1:, :]['CC'].mean(), 6)
                    })
                
                dfSolutions = pd.concat([dfSolutions, pd.DataFrame([entry])], ignore_index=True)

    # Remove duplicate column names if they exist and remove columns with all NA
    dfSolutions = dfSolutions.loc[:, ~dfSolutions.columns.duplicated()]
    dfSolutions = dfSolutions.dropna(axis=1, how='all')


    return dfSolutions

In [23]:
dfSolutionsO1O2O3all = bestModelsLSTM(train_X, train_Y, val_X, val_Y, 
                                         results7alg, list(results7alg.keys()), 
                                         3, 'FS_O1O2O3_LSTM')
dfSolutionsO1O2O3all = calculate_H(dfSolutionsO1O2O3all, config.N_STEPS)
dfSolutionsO1O2O3all

Processing algorithm: NSGAII
Run: 0
Processing algorithm: NSGAIII
Run: 0
Processing algorithm: MOEAD
Run: 0
Processing algorithm: IBEA
Run: 0
Processing algorithm: EpsMOEA
Run: 0
Processing algorithm: SPEA2
Run: 0
Processing algorithm: EpsNSGAII
Run: 0


,Run,Algorithm,RMSE MOEA,N,Correlation,RMSE F Train,RMSE F Test,RMSE StepsAhead,MAE StepsAhead,CC StepsAhead,RMSE MeanStepsAhead,MAE MeanStepsAhead,CC MeanStepsAhead,SelectedAttrib,RMSEnorm,MAEnorm,CCnorm,H
0,0,NSGAII,0.106685,34,-16.634018,0.073876,0.094890,"[0.072809, 0.077897, 0.078997, 0.079166, 0.079...","[0.09489, 0.103481, 0.105035, 0.10541, 0.10556...","[0.39707, 0.28045, 0.261851, 0.255998, 0.25266...",0.081062,0.108233,0.214668,"[Lag_NO_2, Lag_NO_3, Lag_NO_7, Lag_SO2_3, Lag_...",0.081062,0.108233,0.785332,0.324876
1,0,NSGAII,0.075066,47,-19.306887,0.063910,0.081321,"[0.063242, 0.068657, 0.07232, 0.073802, 0.0744...","[0.081321, 0.088204, 0.092706, 0.094546, 0.095...","[0.579228, 0.471349, 0.390686, 0.353926, 0.333...",0.073813,0.094649,0.351597,"[Lag_NO_2, Lag_NO_3, Lag_NO_6, Lag_SO2_1, Lag_...",0.073813,0.094649,0.648403,0.272288
2,0,NSGAII,0.087530,47,-21.862902,0.065039,0.080263,"[0.061764, 0.06978, 0.071127, 0.072598, 0.0733...","[0.080263, 0.090232, 0.092315, 0.094439, 0.095...","[0.595013, 0.455023, 0.423801, 0.389032, 0.369...",0.073679,0.096007,0.365493,"[Lag_NO_1, Lag_NO_2, Lag_NO_3, Lag_NO_4, Lag_N...",0.073679,0.096007,0.634507,0.268064
3,0,NSGAII,0.115931,42,-22.043562,0.066495,0.084847,"[0.065734, 0.067612, 0.067871, 0.067908, 0.067...","[0.084847, 0.087278, 0.087694, 0.087797, 0.087...","[0.555624, 0.52866, 0.523994, 0.521199, 0.5187...",0.068941,0.089730,0.495189,"[Lag_NO_1, Lag_NO_3, Lag_NO_4, Lag_NO_6, Lag_N...",0.068941,0.089730,0.504811,0.221161
4,0,NSGAII,0.119188,38,-20.220342,0.072010,0.083086,"[0.06455, 0.065132, 0.065277, 0.065276, 0.0652...","[0.083086, 0.083762, 0.083913, 0.08397, 0.0840...","[0.555098, 0.545937, 0.544159, 0.541953, 0.539...",0.066198,0.085365,0.518964,"[Lag_NO_1, Lag_NO_5, Lag_NO_7, Lag_SO2_1, Lag_...",0.066198,0.085365,0.481036,0.210866
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,0,EpsNSGAII,0.091254,47,-22.697027,0.066332,0.086265,"[0.066623, 0.069959, 0.07054, 0.070655, 0.0718...","[0.086265, 0.09153, 0.0923, 0.092497, 0.094208...","[0.511788, 0.434559, 0.423611, 0.41943, 0.3922...",0.072198,0.094646,0.386235,"[Lag_NO_1, Lag_NO_2, Lag_NO_3, Lag_NO_7, Lag_S...",0.072198,0.094646,0.613765,0.260203
137,0,EpsNSGAII,0.153203,43,-23.552576,0.065520,0.080564,"[0.06236, 0.070576, 0.074095, 0.075517, 0.0769...","[0.080564, 0.090462, 0.09574, 0.09815, 0.10057...","[0.596805, 0.462273, 0.387255, 0.352475, 0.318...",0.076578,0.099827,0.330993,"[Lag_NO_1, Lag_NO_2, Lag_NO_3, Lag_NO_4, Lag_N...",0.076578,0.099827,0.669007,0.281804
138,0,EpsNSGAII,0.077771,41,-20.719330,0.066184,0.077094,"[0.060026, 0.06827, 0.071954, 0.073715, 0.0738...","[0.077094, 0.086674, 0.090723, 0.092575, 0.092...","[0.644396, 0.507872, 0.437562, 0.401603, 0.394...",0.073419,0.092388,0.404554,"[Lag_NO_1, Lag_NO_4, Lag_NO_5, Lag_SO2_1, Lag_...",0.073419,0.092388,0.595446,0.253751
139,0,EpsNSGAII,0.093204,38,-18.168292,0.067884,0.078091,"[0.059128, 0.068162, 0.070016, 0.072291, 0.073...","[0.078091, 0.088602, 0.091234, 0.094145, 0.095...","[0.628838, 0.498622, 0.464945, 0.425557, 0.402...",0.072756,0.094974,0.414202,"[Lag_NO_1, Lag_NO_2, Lag_NO_3, Lag_NO_4, Lag_N...",0.072756,0.094974,0.585798,0.251176


In [ ]:
# Save results
with open(rf'../Variables/{config.DATASET_SAVE_NAME}-dfSolutions-LSTM-3obj-O1O2O3-all.pickle', 'wb') as f:
     pickle.dump([dfSolutionsO1O2O3all], f)

In [18]:
idxBestRMSEF = dfSolutionsO1O2O3all.groupby(['Run', 'Algorithm'])['H'].transform("min") == dfSolutionsO1O2O3all['H']
dfSolutionsO1O2O3all[idxBestRMSEF].sort_values(by='H')

,Run,Algorithm,RMSE MOEA,N,Correlation,RMSE F Train,RMSE F Test,RMSE StepsAhead,MAE StepsAhead,CC StepsAhead,RMSE MeanStepsAhead,MAE MeanStepsAhead,CC MeanStepsAhead,SelectedAttrib,RMSEnorm,MAEnorm,CCnorm,H
104,0,SPEA2,0.119188,38,-20.220342,0.067647,0.082459,"[0.063937, 0.064996, 0.065101, 0.065048, 0.065...","[0.082459, 0.083229, 0.083356, 0.083365, 0.083...","[0.590197, 0.583878, 0.583501, 0.583496, 0.583...",0.066189,0.084274,0.575211,"[Lag_NO_1, Lag_NO_5, Lag_NO_7, Lag_SO2_1, Lag_...",0.066189,0.084274,0.424789,0.191751
79,0,EpsMOEA,0.115931,42,-22.043562,0.065769,0.080397,"[0.061345, 0.062323, 0.062505, 0.062508, 0.062...","[0.080397, 0.081458, 0.081667, 0.081728, 0.081...","[0.594515, 0.58169, 0.579516, 0.57789, 0.57622...",0.063034,0.082561,0.565476,"[Lag_NO_1, Lag_NO_3, Lag_NO_4, Lag_NO_6, Lag_N...",0.063034,0.082561,0.434524,0.193373
37,0,NSGAIII,0.079755,37,-18.500277,0.066666,0.071516,"[0.054475, 0.061531, 0.064096, 0.064837, 0.065...","[0.071516, 0.079641, 0.082811, 0.083783, 0.084...","[0.70056, 0.606327, 0.563635, 0.548783, 0.5408...",0.065210,0.084372,0.539194,"[Lag_NO_2, Lag_NO_6, Lag_SO2_1, Lag_SO2_3, Lag...",0.065210,0.084372,0.460806,0.203463
55,0,IBEA,0.115931,42,-22.043562,0.065776,0.083369,"[0.063656, 0.064487, 0.06473, 0.064824, 0.0649...","[0.083369, 0.08428, 0.08445, 0.084567, 0.08467...","[0.565377, 0.554496, 0.552826, 0.550549, 0.548...",0.065845,0.085657,0.536995,"[Lag_NO_1, Lag_NO_3, Lag_NO_4, Lag_NO_6, Lag_N...",0.065846,0.085656,0.463006,0.204836
3,0,NSGAII,0.115931,42,-22.043562,0.068696,0.081937,"[0.063445, 0.064816, 0.065053, 0.065082, 0.065...","[0.081937, 0.083335, 0.083586, 0.08367, 0.0837...","[0.574868, 0.557721, 0.554884, 0.552705, 0.550...",0.065870,0.084886,0.535059,"[Lag_NO_1, Lag_NO_3, Lag_NO_4, Lag_NO_6, Lag_N...",0.065870,0.084886,0.464941,0.205232
132,0,EpsNSGAII,0.109669,35,-17.342835,0.068763,0.078888,"[0.061743, 0.063894, 0.064204, 0.064163, 0.065...","[0.078888, 0.081959, 0.082349, 0.082376, 0.083...","[0.617664, 0.575294, 0.569786, 0.568067, 0.542...",0.065801,0.084433,0.534102,"[Lag_NO_1, Lag_NO_3, Lag_NO_5, Lag_SO2_2, Lag_...",0.065801,0.084433,0.465898,0.205377
42,0,MOEAD,0.157569,36,-18.138793,0.068953,0.073151,"[0.055264, 0.063257, 0.066991, 0.069553, 0.070...","[0.073151, 0.082664, 0.088244, 0.091392, 0.093...","[0.684196, 0.579229, 0.51106, 0.47151, 0.44369...",0.070355,0.093152,0.451726,"[Lag_NO_1, Lag_NO_2, Lag_NO_4, Lag_NO_5, Lag_N...",0.070355,0.093153,0.548275,0.237261


## Test results for best FS

In [10]:
# Load
with open(rf'../Variables/{config.DATASET_SAVE_NAME}-dfSolutions-LSTM-3obj-O1O2O3-all.pickle', 'rb') as f:
    dfSolutionsO1O2O3all= pickle.load(f)[0]

In [20]:
# Store the best attribute combination per algorithm
for alg in np.unique(dfSolutionsO1O2O3all['Algorithm']):
    get_dataset_best_H(dataset, dfSolutionsO1O2O3all[dfSolutionsO1O2O3all['Algorithm'] == alg]).to_csv(rf"./data/{config.DATASET_SAVE_NAME}-O1O2O3-{alg}.csv", index=False)

In [21]:
# Define algorithm names
algorithms = ["EpsMOEA", "EpsNSGAII", "IBEA", "MOEAD", "NSGAII", "NSGAIII", "SPEA2"]

# Load datasets and preprocess
df_dicts = {
    algo: preprocess_data(pd.read_csv(rf"./data/{config.DATASET_SAVE_NAME}-O1O2O3-{algo}.csv"))
    for algo in algorithms
}

# Extract normalized data
normalized_data = {algo: df_dicts[algo]['normalized'] for algo in algorithms}

# Perform training, validation, and testing
random.seed(config.SEED_VALUE)
np.random.seed(config.SEED_VALUE)
tf.random.set_seed(config.SEED_VALUE)

df_results = {algo: train_evaluate_lstm_model(*normalized_data[algo], config.N_NEURONS, config.BATCH_SIZE, config.EPOCHS, config.N_STEPS) for algo in algorithms}

# Concatenate results
dfTrainValTest = pd.concat(df_results.values(), ignore_index=True)

# Assign algorithm labels
dfTrainValTest['Algorithm'] = list(df_results.keys())

dfTrainValTest

,RMSE StepsAhead Train,MAE StepsAhead Train,CC StepsAhead Train,H Train,RMSE StepsAhead Val,MAE StepsAhead Val,CC StepsAhead Val,H Val,RMSE StepsAhead Test,MAE StepsAhead Test,CC StepsAhead Test,H Test,Algorithm
0,"[0.047801, 0.048379, 0.047966, 0.047882, 0.047...","[0.066763, 0.067314, 0.065862, 0.065765, 0.065...","[0.917781, 0.916337, 0.917852, 0.917991, 0.917...",0.066504,"[0.062566, 0.063732, 0.063924, 0.063933, 0.063...","[0.081425, 0.082636, 0.082831, 0.082897, 0.082...","[0.583801, 0.567743, 0.565672, 0.563968, 0.562...",0.199571,"[0.05454, 0.055418, 0.055527, 0.055082, 0.0547...","[0.069257, 0.07018, 0.070315, 0.069606, 0.0691...","[0.709019, 0.701141, 0.700347, 0.709955, 0.716...",0.139110,EpsMOEA
1,"[0.047775, 0.048644, 0.048488, 0.048323, 0.048...","[0.066448, 0.067427, 0.065996, 0.065728, 0.066...","[0.917748, 0.91537, 0.916861, 0.917536, 0.9143...",0.067833,"[0.063073, 0.065594, 0.0659, 0.065882, 0.06645...","[0.080908, 0.084191, 0.084603, 0.084659, 0.085...","[0.58739, 0.541739, 0.536489, 0.534955, 0.5256...",0.212722,"[0.062377, 0.065845, 0.066264, 0.065959, 0.066...","[0.075784, 0.079941, 0.080415, 0.079997, 0.080...","[0.680831, 0.630804, 0.625252, 0.631214, 0.627...",0.178293,EpsNSGAII
2,"[0.047235, 0.048225, 0.047808, 0.047769, 0.047...","[0.066366, 0.067247, 0.065829, 0.065803, 0.065...","[0.916661, 0.914178, 0.916015, 0.91574, 0.9149...",0.067221,"[0.065362, 0.067101, 0.067392, 0.067395, 0.067...","[0.084023, 0.086106, 0.08646, 0.08653, 0.08657...","[0.554264, 0.529041, 0.52487, 0.522622, 0.5206...",0.219527,"[0.059323, 0.062167, 0.062538, 0.062176, 0.061...","[0.073056, 0.075872, 0.076251, 0.075729, 0.075...","[0.700111, 0.678088, 0.675478, 0.681317, 0.685...",0.155042,IBEA
3,"[0.049582, 0.055169, 0.057573, 0.059199, 0.059...","[0.070945, 0.078302, 0.081172, 0.083723, 0.084...","[0.906937, 0.887237, 0.87575, 0.867625, 0.8636...",0.092148,"[0.058318, 0.065926, 0.069315, 0.070958, 0.072...","[0.076378, 0.085672, 0.090604, 0.093021, 0.094...","[0.643407, 0.51811, 0.441123, 0.400025, 0.3701...",0.263451,"[0.055241, 0.063245, 0.067313, 0.069027, 0.069...","[0.068464, 0.076986, 0.081791, 0.083615, 0.084...","[0.76629, 0.678683, 0.617911, 0.594759, 0.5874...",0.186474,MOEAD
4,"[0.048189, 0.048832, 0.048409, 0.048322, 0.048...","[0.066794, 0.067394, 0.065928, 0.065828, 0.065...","[0.918466, 0.916856, 0.918463, 0.918566, 0.917...",0.066485,"[0.06742, 0.068385, 0.068617, 0.068699, 0.0687...","[0.088432, 0.089583, 0.089805, 0.089923, 0.090...","[0.482258, 0.468092, 0.465671, 0.463015, 0.460...",0.242315,"[0.049394, 0.050062, 0.050223, 0.049877, 0.049...","[0.065084, 0.066218, 0.066397, 0.065922, 0.065...","[0.722399, 0.708511, 0.707218, 0.713099, 0.719...",0.134431,NSGAII
5,"[0.045926, 0.049752, 0.050966, 0.051782, 0.051...","[0.064089, 0.069222, 0.070609, 0.071589, 0.071...","[0.922992, 0.910012, 0.903687, 0.900472, 0.899...",0.074361,"[0.057584, 0.064077, 0.067399, 0.068155, 0.068...","[0.075117, 0.083048, 0.087384, 0.088686, 0.089...","[0.660781, 0.56089, 0.497037, 0.475748, 0.4639...",0.231998,"[0.059506, 0.066281, 0.069687, 0.070271, 0.070...","[0.074065, 0.082055, 0.08597, 0.086552, 0.0865...","[0.676267, 0.564718, 0.508566, 0.501318, 0.502...",0.220822,NSGAIII
6,"[0.04903, 0.049206, 0.048644, 0.048527, 0.0485...","[0.069268, 0.069841, 0.067411, 0.067253, 0.067...","[0.909084, 0.907377, 0.911896, 0.912056, 0.911...",0.069593,"[0.067851, 0.068365, 0.068478, 0.068438, 0.068...","[0.08567, 0.086109, 0.086249, 0.08627, 0.08628...","[0.528577, 0.526088, 0.525223, 0.52473, 0.5242...",0.217719,"[0.057408, 0.057819, 0.057965, 0.057632, 0.057...","[0.073083, 0.073678, 0.07381, 0.073382, 0.0730...","[0.656884, 0.649723, 0.648822, 0.654517, 0.658...",0.162644,SPEA2
